In [ ]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta, time
from research.utils.data_access_utils import RDSAccessUtils
from research.weight_estimation.keypoint_utils.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator

rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

In [ ]:
queryCache = {}

In [ ]:
pen5 = pd.read_csv('blom_vikane_singleweights.csv')

In [ ]:
pen5['weight']

In [ ]:
plt.hist(pen5['weight'], bins = 50)

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

def getPenDF(pen):
    akpd_filter = 0.99

    query = """
       SELECT * FROM prod.biomass_computations
        WHERE prod.biomass_computations.captured_at >= '%s'
        AND prod.biomass_computations.captured_at <= '%s'
        AND prod.biomass_computations.akpd_score > %0.4f
        AND prod.biomass_computations.pen_id = %i;
    """ % (pen['start_date'], pen['end_date'], akpd_filter, pen['pen_id'])

    if query in queryCache:
        df = queryCache[query].copy()
    else:
        df = rds_access_utils.extract_from_database(query)
        queryCache[query] = df.copy()

    df = df.sort_values('captured_at').copy(deep=True)
    df.index = pd.to_datetime(df.captured_at)
    dates = df.index.date.astype(str)
    df['date'] = dates
    df['hour'] = df.index.hour
    
    return df

def getNegativeWeightDF():
    akpd_filter = 0.99

    query = """
       SELECT * FROM prod.biomass_computations
        WHERE prod.biomass_computations.estimated_weight_g < 0
        AND prod.biomass_computations.akpd_score > %0.4f;
    """ % (akpd_filter, )

    if query in queryCache:
        df = queryCache[query].copy()
    else:
        df = rds_access_utils.extract_from_database(query)
        queryCache[query] = df.copy()

    df = df.sort_values('captured_at').copy(deep=True)
    df.index = pd.to_datetime(df.captured_at)
    dates = df.index.date.astype(str)
    df['date'] = dates
    df['hour'] = df.index.hour
    
    return df

pens = [
    {
        'pen_id': 60,
        'start_date': '2020-08-24 00:00',
        'end_date': '2020-08-26 00:00'
    }
]

df1 = getPenDF(pens[0])
df2 = getNegativeWeightDF()

In [ ]:
print(len(df1))

In [ ]:
plt.figure(figsize=(20, 10))
s1, a1 = np.mean(pen5['weight'] * 1000), np.mean(df1.estimated_weight_g), 
plt.hist(df1.estimated_weight_g, density = True, alpha = 0.5, label = 'Aquabyte', bins = 20)
plt.hist(pen5.weight * 1000 / 0.83, density = True, alpha = 0.5, label = 'Sampling', bins = 20)
plt.legend()
plt.title('Pen 5: Sampling (%i) vs Aquabyte (%i)' % (s1, a1))

In [ ]:
low_weight = df1[df1.estimated_weight_g < 1000]
low_weight.ix[0]

In [ ]:
index = 0

from IPython.display import Image
from IPython.core.display import HTML 
Image(url=low_weight.ix[index, 'left_crop_url'])

In [ ]:
Image(url=low_weight.ix[index, 'right_crop_url'])

In [ ]:
row1 = low_weight.ix[0]
ann1, cm1 = row1.annotation, row1.camera_metadata
wkps1 = pixel2world(ann1['leftCrop'], ann1['rightCrop'], cm1)
wkps1

In [ ]:
index = -2

from IPython.display import Image
from IPython.core.display import HTML 
Image(url=df2.ix[index, 'left_crop_url'])

In [ ]:
Image(url=df2.ix[index, 'right_crop_url'])

In [ ]:
plt.figure(figsize=(20, 10))

im = plt.imread('left_frame_crop_1628_2136_3788_3000.jpg')
implot = plt.imshow(im)

for ann in df2.ix[-1].annotation['leftCrop']:
    plt.scatter([ann['xCrop']], [ann['yCrop']])
    plt.text(ann['xCrop'], ann['yCrop'], ann['keypointType'], color = 'white')

plt.show()

In [ ]:
plt.figure(figsize=(20, 10))

im = plt.imread('right_frame_crop_772_2124_3100_3000.jpg')
implot = plt.imshow(im)

for ann in df2.ix[-1].annotation['rightCrop']:
    plt.scatter([ann['xCrop']], [ann['yCrop']])
    plt.text(ann['xCrop'], ann['yCrop'], ann['keypointType'], color = 'white')

plt.show()

In [ ]:
row1 = df2.ix[-1]
ann1, cm1 = row1.annotation, row1.camera_metadata
wkps1 = pixel2world(ann1['leftCrop'], ann1['rightCrop'], cm1)
wkps1

In [ ]:
np.linalg.norm(wkps1['UPPER_LIP'] - wkps1['TAIL_NOTCH'])

In [ ]:
row1 = df2.ix[-1]
ann1, cm1 = row1.annotation, row1.camera_metadata
wkps1 = pixel2world(ann1['leftCrop'], ann1['rightCrop'], cm1)
wkps1

In [ ]:
vector = wkps1['PECTORAL_FIN'] - wkps1['ANAL_FIN']
x, y, z = vector / np.linalg.norm(vector)
print(x, y, z)

In [ ]:
import math 
# https://www.mathworks.com/matlabcentral/answers/108850-what-would-the-angles-be-for-a-3d-vector
ax = math.atan2(np.sqrt(y ** 2 + z ** 2), x)
ay = math.atan2(np.sqrt(z ** 2 + x ** 2), y)
az = math.atan2(np.sqrt(x ** 2 + y ** 2), z)

ax = ma

dx = math.degrees(ax)
dy = math.degrees(ay)
dz = math.degrees(az)

ax, ay, az
dx, dy, dz

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

soa = np.array([[0, 0, 0, x, y, z]])

X, Y, Z, U, V, W = zip(*soa)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.quiver(X, Y, Z, U, V, W)
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()